Técnicas de machine learning aplicadas a la predicción de precios de stocks. 

In [1]:
# Manipulación de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Plots

# Keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers.core import Dense,Activation,Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
import warnings
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from math import sqrt

#Manejo de archivos
import os
import tempfile
import shutil

c:\users\leandro\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
class Data:
    "Recibe una lista de nombres archivos csv de stocks y administra sus datos."
    __dir_data = "data"
    
    def __init__(self,stock_names,fecha_comienzo,fecha_fin):
        self.stock_names = stock_names
        self.fecha_comienzo = fecha_comienzo
        self.fecha_fin = fecha_fin
        
        stockdata = self.__get_stocks()
        
        # Seleccionar los datos dentro del rango de fechas
        self.data = pd.DataFrame(pd.date_range(start=self.fecha_comienzo, end=self.fecha_fin, freq='D'),columns=['Date'])
        self.data.set_index('Date', inplace=True)
        
        # Hacer join de los dataframes (fechas - precios) para tener precios para las fechas deseadas
        self.data = pd.merge(self.data, stockdata,  how='left', left_index=True, right_index=True)
        self.data.replace('-',np.nan,inplace=True) 
        self.data = self.data.interpolate().ffill().fillna(0) #Completar campos faltantes
    
        # Guardar Datos
        if not os.path.exists(self.__dir_data):
            os.makedirs(self.__dir_data)
        path = self.__dir_data + "/" + 'data' + '.csv'
        self.data.to_csv(path, index=True)
        print('Datos recolectados, archivo: ' + path)
    
    def __get_stocks(self):    
        "Arma un dataframe con todos los datos de los archivos de stocks"
        stock_dataframe =  pd.DataFrame()
        
        for stock in self.stock_names:
            df = pd.read_csv('data/original/' + stock + '.csv')

            df.drop(['Open','High','Low','Adj Close','Volume'], axis=1, inplace=True)
            df.rename(columns={'Close': stock}, inplace=True)
            df.set_index('Date', inplace=True)
    
            stock_dataframe = pd.concat([stock_dataframe, df], axis=1)

            print('Stock procesado: ' + stock)
        
        stock_dataframe.index.name = 'Date'
        
        return stock_dataframe
    
    def plot(self):
        "Graficar los datos del DataManager"
        # Graficar TODAS las columnas juntas
        plt.figure(figsize=(20,5))
        plt.title("Stock Prices", fontsize=25)
        for stock in np.arange(0, len(self.data.columns), 1):
            plt.plot(self.data[self.data.columns[stock]], label=self.data.columns[stock])    
        major_ticks = np.arange(0, len(self.data), 30)
        plt.xticks(major_ticks, np.array(self.data.index[::30]), rotation=20)
        plt.legend()
        plt.show()
        
        # Graficar cada columna
        i = 1
        # plot each column
        plt.figure(figsize=(20,5*len(self.data.columns)))
        for stock in np.arange(0, len(self.data.columns), 1):
            plt.subplot(len(self.data.columns), 1, i)
            plt.plot(self.data[self.data.columns[stock]], label=self.data.columns[stock])   
            major_ticks = np.arange(0, len(self.data), 30)
            plt.xticks(major_ticks, np.array(self.data.index[::30]), rotation=20)
            plt.legend()
            i+=1
        plt.show()
        
    def create_dataset(self,stock,fecha_comienzo,fecha_fin):
        """
        Convierte el conjunto de datos en dos conjuntos X e Y (input/output)
 
        :param moneda: nombre de la columna en el dataframe del target
        :returns: conjuntos X e Y como numpy array
        """
        # Seleccionar los datos dentro del rango de fechas
        dataset = pd.DataFrame(pd.date_range(start=fecha_comienzo, end=fecha_fin, freq='D'),columns=['Date'])
        dataset.set_index('Date', inplace=True)
        dataset = pd.merge(dataset, dm.data,  how='left', left_index=True, right_index=True)
        dataset = dataset.interpolate().ffill().fillna(0) #Completar campos faltantes
        
        pos_y = dataset.columns.get_loc(stock) # Obtener columna correspondiente al stock a predecir
        dataset = dataset.values
        x = np.asarray(dataset)
        y = np.asarray(dataset[:,pos_y])
        
        return {'x': x, 'y': y, 'stock':stock}

In [6]:
stocks_names = ['AAPL','FB','GOOG','GOOGL','IBM',
                'INTC','MSFT','NFLX','ORCL','TSLA']
stockindex_names = ['^IXIC','^RUT','^DJI','^GSPC']

# DataManager
dm = Data(stock_names=stockindex_names,fecha_comienzo='2017-01-01', fecha_fin='2018-05-25')
print(dm.data)

Stock procesado: ^IXIC
Stock procesado: ^RUT
Stock procesado: ^DJI
Stock procesado: ^GSPC
Datos recolectados, archivo: data/data.csv
                  ^IXIC         ^RUT          ^DJI        ^GSPC
Date                                                           
2017-01-01     0.000000     0.000000      0.000000     0.000000
2017-01-02     0.000000     0.000000      0.000000     0.000000
2017-01-03  5429.080078  1365.489990  19881.759766  2257.830078
2017-01-04  5477.000000  1387.949951  19942.160156  2270.750000
2017-01-05  5487.939941  1371.939941  19899.289063  2269.000000
2017-01-06  5521.060059  1367.280029  19963.800781  2276.979980
2017-01-07  5524.646647  1364.016683  19938.327474  2274.286621
2017-01-08  5528.233236  1360.753336  19912.854166  2271.593261
2017-01-09  5531.819824  1357.489990  19887.380859  2268.899902
2017-01-10  5551.819824  1370.900024  19855.529297  2268.899902
2017-01-11  5563.649902  1373.300049  19954.279297  2275.320068
2017-01-12  5547.490234  1361.06994